# Find real last sensor change, ignore HA restart

This is based on https://community.home-assistant.io/t/real-state-last-changed/15928 but using InfluxDB/SQLite

# InfluxDB

In [2]:
import sys
from influxdb import DataFrameClient
from datetime import timezone

entity_id = sys.argv[1]

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

client = DataFrameClient('192.168.1.199', 8086, '', '', 'home_assistant')

InfluxDB query:

```sql
select * from "°C" where domain='sensor' and entity_id='attic_office_temperature' order by time DESC limit 10
```

Note that this still shows restart times as it looks like HA sets the sensor values into influxdb when it starts. Need to loop through results to find when value actually changed :|

This isn't very flexible however as it requires knowing what the sensor unit is, to access the requisite table.

## SQLite

Use the built-in database (careful!) to extract the necessary data

In [64]:
# todo: error handling, think about utc

import sys
from sqlalchemy import create_engine
import pandas as pd

# define your home assistant db location below:
db = "/home/amadeus/.homeassistant/home-assistant_v2.db"

# specify entity_id as first argument to this script
entity_id = "sensor.attic_office_temperature"
# entity_id = sys.argv[1]

# connect using SQLalchemy
engine = create_engine(f"sqlite:///{db}")
conn = engine.connect()

# how many restart events can we expect?
num_restarts = 100

# build the query
query = f"""
SELECT
    state,
    last_changed
FROM states 
WHERE 
    domain="sensor" AND
    entity_id="{entity_id}" AND
    state !="unknown"
ORDER BY last_updated DESC 
LIMIT {num_restarts}
"""

# execute the query and load into a pandas dataframe:
df = pd.read_sql_query(query, conn)

# compare the results to a dataframe shifted one forward with backfill,
# extract the 1st changed value, 
# and extract the last_changed value

real_last_changed = pd.to_datetime(df.last_changed[df.state.ne(df.state.shift().bfill())].iloc[0])

Implementation in hassio:

```yaml
sensor:
  - platform: command_line
    name: "RLC MQTT Test Switch"
    command: "python3 /path/to/scripts/real_last_change.py sensor.attic_office_temperature"
    scan_interval: 60
```

Template to calculate when sensor was last changed (hours) for automations:

```yaml
{{ (as_timestamp(now()) - as_timestamp(states('sensor.rlc_attic_office_temperature')))/(60*60) }}
```